<img src="imgs/hpe_logo.png" alt="HPE Logo" width="300">

# Chicago DS Summit Workshop: Chatbot tutorial: Finetuning GPT models at Scale with Machine Learning Development Environment
 ----

Note this Demo is based on https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm_no_trainer.py

## Objective: Train your own chatbot
This notebook walks you through finetuning your own chatbot.
We will learn what are Generative Pretrained Transformers (GPT), how to finetune to your own domain, and finetune at Scale

## Why is this exciting: The rise of generative language modeling
Generative Language models like GPT-4 and ChatGPT enable exciting applications that were not possible before!

* `Enterprise`: Chatbots for helpdesk support
* `Healthcare`: Chatbots for scheduling appts, manage coverage, process claims
* `Manufacturing`: Chatbots for checking supplies and inventory check
* `Financial Services`: Chatbots for investment and account support

With Machine Learning Development Environment (MLDE, based on DeterminedAI) we help engineers create powerful language modeling application at scale!

## Why MLDE and DeterminedAI

Developing robust, high performing Deep Learning application is challenging. Succesful team requires data, compute, and great infrastructure. Building and managing distributed training, automatic checkpointing, hyperparameter search and metrics tracking is critical. 

MLDE can remove the burden of writing and maintaining a custom training harness and offers a streamlined approach to onboard new models to a state-of-the-art training platform, offering the following integrated platform features:

<img src="imgs/det_components.jpg" alt="Determined Components" width="900">

Determined provides a high-level framework APIs for PyTorch, Keras, and Estimators that let users describe their model without boilerplate code. Determined reduces boilerplate by providing a state-of-the-art training loop that provides distributed training, hyperparameter search, automatic mixed precision, reproducibility, and many more features.

<h3>Overview of this workshop</h3>

## Overview of Tutorial

* Intro to GPT models
* Data Science Chatbot Baseline: Chatbot to ask Data Science questions
* Finetune GPT model on a Data Science book to better answer Data Science Tasks
* Overview of integrating Pytorch training code into MLDE
* English to Latex Chatbot: Convert english to latex
* Finetune GPT model on dataset to convert english to latex
* User exercise: Finetune bot on custom text data

Let's get started!

---

In [3]:
from transformers import GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         Trainer, TrainingArguments
import torch
from datasets import Dataset
import pandas as pd
from determined.experimental import Determined
from utils import load_model_from_checkpoint
from determined.experimental import client
from determined import pytorch

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# What is GPT

<img src="imgs/openAI-gpt2.png" alt="Determined Components" width="900">

GPT2 is a transformer-based language model created and released by OpenAI. The model is a causal (unidirectional) transformer pre-trained using language modeling on a large corpus with long range dependencies.

Other models available: GPT2-Medium, GPT2-Large and GPT2-XL

# Data Science Chatbot

## Baseline
We will load a pretrained model on X dataset and see how it responds to data science questions.

Prompt: `A test statistic is a value`

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')  # load up a GPT2 model
pretrained_generator = pipeline(
    'text-generation', model=model, tokenizer='gpt2',
    config={'max_length': 200, 'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

In [ ]:
PROMPT='A test statistic is a value'

In [ ]:

print('----------')
for generated_sequence in pretrained_generator(PROMPT, num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/opt/conda/lib/python3.8/site-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


----------
A test statistic is a value that shows a change in a certain probability value.

To put it simply, if this statistic changes, then the test statistic gets a change of 0.0 from the last time you saw it. Then when you
----------
A test statistic is a value that makes little any sense.
1.1 Test Number Test Number to be verified


The results from this test match the 1st Test Average. The 1st Test Average matches that the 100 second average was at
----------
A test statistic is a value between 1 and 50. This test statistic can be checked at any time by simply viewing the sample, and the value is then multiplied by 50.
----------


### What model training looks like without Determined

```python

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='./data/PDS2.txt',  # Principles of Data Science - Sinan Ozdemir
    block_size=32  # length of each chunk of text to use as a datapoint
)
config = GPT2Config.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

train_sampler = RandomSampler(dataset)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
train_dataloader = DataLoader(dataset, collate_fn =data_collator ,sampler=train_sampler, batch_size=train_batch_size)

t_total = len(dataset) // gradient_accumulation_steps * num_train_epochs
# Prepare optimizer and schedule (linear warmup and decay)
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

model.resize_token_embeddings(len(tokenizer))
model.zero_grad()
train_iterator = trange(int(num_train_epochs), desc="Epoch", disable=local_rank not in [-1, 0])
set_seed(0)
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=local_rank not in [-1, 0])
    for step, batch in enumerate(epoch_iterator):
        inputs, labels = mask_tokens(batch, tokenizer, None) if mlm else (batch, batch)
        inputs = inputs.to(device)
        labels = labels.to(device)
        model.train()

        outputs = model(inputs, masked_lm_labels=labels) if mlm else model(inputs, labels=labels)
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
        if fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        loss.backward()
```

### Here is what it would look like to do hyperparameter search

```python

import numpy as np

def train(lr,m):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path='./data/PDS2.txt',  # Principles of Data Science - Sinan Ozdemir
        block_size=32  # length of each chunk of text to use as a datapoint
    )
    config = GPT2Config.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model.to(device)
    train_sampler = RandomSampler(dataset)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    train_dataloader = DataLoader(dataset, collate_fn =data_collator ,sampler=train_sampler, batch_size=train_batch_size)

    t_total = len(dataset) // gradient_accumulation_steps * num_train_epochs

    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

    model.resize_token_embeddings(len(tokenizer))
    model.zero_grad()
    train_iterator = trange(int(num_train_epochs), desc="Epoch", disable=local_rank not in [-1, 0])
    set_seed(0)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            inputs, labels = mask_tokens(batch, tokenizer, None) if mlm else (batch, batch)
            inputs = inputs.to(device)
            labels = labels.to(device)
            model.train()

            outputs = model(inputs, masked_lm_labels=labels) if mlm else model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
            if fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            loss.backward()
    model, loss
def hp_grid_search():
    for lr in np.logspace(-4, -2, num=10):
        for m in np.linspace(0.7, 0.95, num=10):
            print(f"Training model with learning rate {lr} and momentum {m}")
            model, loss = train(lr,m)
            print(f"Train Loss: {loss}\n")

try:
    hp_grid_search()
except KeyboardInterrupt:
    pass


```

#### What's Missing?
This approach works in theory -- we could get a good model, save it, and use it for predictions. But we're missing a lot from the ideal state:

#### Distributed training
    - Parallel search
    - Intelligent checkpointing
    - Interruptibility and fault tolerance
    - Logging of experiment configurations and results

# Overview of integrating Pytorch training code into MLDE

The main components for any deep learning training loop are the following:
* Datasets
* Dataloader
* Model
* Optimizer
* (Optional) Learn rate schedule
* training a batch, evaluating a batch

We will show how to integrate each core part into MLDE using the PyTorchTrial API. Note we have another API called CoreAPI, that supports flexibility if your team wants to integrate more complex Machine Learning codebases. 

### Template Class that integrates DL code with MLDE

```python
import filelock
import os
from typing import Any, Dict, Sequence, Tuple, Union, cast

import torch
import torch.nn as nn
from torch import optim
from determined.pytorch import DataLoader, PyTorchTrial, PyTorchTrialContext

import data

TorchData = Union[Dict[str, torch.Tensor], Sequence[torch.Tensor], torch.Tensor]

class GPT2Trial(PyTorchTrial):
    def __init__(self, context: PyTorchTrialContext) -> None:
        # Trial context contains info about the trial, such as the hyperparameters for training
        self.context = context
        
        # init and wrap model, optimizer, LRscheduler, datasets
       

    def build_training_data_loader(self) -> DataLoader:
        # create train dataloader from dataset
        return DataLoader()

    def build_validation_data_loader(self) -> DataLoader:
        # create train dataloader from dataset
        return DataLoader()

    def train_batch(self, batch: TorchData, epoch_idx: int, batch_idx: int)  -> Dict[str, Any]:
        return {}

    def evaluate_batch(self, batch: TorchData) -> Dict[str, Any]:
        return {}
```

### Wrap Model
* Wrapping model to the TrialContext allows MLDE to reduces boilerplate code
* Providing a state-of-the-art training loop that provides distributed training, hyperparameter search, automatic mixed precision, reproducibility, and many more features
* All the models, optimizers, and LR schedulers must be wrapped with wrap_model and wrap_optimizer respectively

```python
self.model = GPT2LMHeadModel.from_pretrained('gpt2')
# Wrapping model to the TrialContext 
self.model = self.context.wrap_model(self.model)
```

### Wrap Optimizer

```python
self.optimizer = self.context.wrap_optimizer(
            AdamW(optimizer_grouped_parameters, lr=self.learning_rate, eps=self.adam_epsilon)
            )
```

### Wrap LR scheduler

```python
self.scheduler = self.context.wrap_lr_scheduler(
    get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps=self.warmup_steps,
                                    num_training_steps=self.t_total),
    LRScheduler.StepMode.MANUAL_STEP
)
```

### Integrate Dataset

```python
dataset = TextDataset(
                tokenizer=tokenizer,
                file_path='/run/determined/workdir/shared_fs/workshop_data/hamlet.txt',
                block_size=32  # length of each chunk of text to use as a datapoint
            )
```

### Implement Train Dataloader and Validation Dataloader

```python
def build_training_data_loader(self) -> None:
    '''
    '''
    self.train_sampler = RandomSampler(self.dataset)
    self.train_dataloader = DataLoader(self.dataset, collate_fn =self.data_collator ,sampler=self.train_sampler, batch_size=self.train_batch_size)
    return self.train_dataloader
def build_validation_data_loader(self) -> None:
    '''
    '''
    self.eval_sampler = SequentialSampler(self.dataset)
    self.validataion_dataloader = DataLoader(self.dataset,collate_fn =self.data_collator, sampler=self.eval_sampler, batch_size=self.eval_batch_size)
```

### Implement Train Batch and Evaluate Batch

```python
def train_batch(self,batch,epoch_idx, batch_idx):
    '''
    '''
    inputs,labels = self.format_batch(batch)
    outputs = self.model(inputs, labels=labels)
    loss = outputs[0]
    train_result = {
        'loss': loss
    }
    self.context.backward(train_result["loss"])
    self.context.step_optimizer(self.optimizer)
    return train_result

def evaluate_batch(self,batch):
    '''
    '''
    inputs,labels = self.format_batch(batch)
    outputs = self.model(inputs, labels=labels)
    lm_loss = outputs[0]
    eval_loss = lm_loss.mean().item()
    perplexity = torch.exp(torch.tensor(eval_loss))

    results = {
        "eval_loss": eval_loss,
        "perplexity": perplexity
    }
    return results
```

# Defining Training Experiment with Model Config
In Determined, a trial is a training task that consists of a dataset, a deep learning model, and values for all of the model’s hyperparameters. An experiment is a collection of one or more trials: an experiment can either train a single model (with a single trial), or can define a search over a user-defined hyperparameter space.


Here is what a configuration file looks like for a single trial experiment
```yaml
name: gpt2_finetune_data_science_chatbot
workspace: <your_workspace>
project: <your_project>
description: "Chicago DS Workshop"
hyperparameters:
    global_batch_size: 32
    weight_decay: 0.0
    learning_rate: 5e-5
    adam_epsilon: 1e-8
    warmup_steps: 0
    epochs: 10
    gradient_accumulation_steps: 1
    dataset_name: 'PDS2'
environment:
    image: "hugcyrill/workshops:chat_0.1"
records_per_epoch: 147 # 4696 examples, 128 per batch: 601088/21 is 147 records for an epoch
resources:
    slots_per_trial: 1
min_validation_period:
  batches: 4
min_checkpoint_period:
  batches: 147
searcher:
    name: single
    metric: eval_loss
    max_length:
        epochs: 30
    smaller_is_better: true
max_restarts: 0
entrypoint: model_def:GPT2Finetune
```

Here is what a configuration yaml file looks like to do a hyperparameter search
```yaml
name: adaptive_gpt2_finetune_data_science_chatbot
workspace: <your_workspace>
project: <your_project>
description: "Chicago DS Workshop"
hyperparameters:
    global_batch_size: 32
    weight_decay: 0.0
    learning_rate:
        type: log
        minval: -6.0
        maxval: -4.0
        base: 10.0
    adam_epsilon:
        type: log
        minval: -10.0
        maxval: -4.0
        base: 10.0
    warmup_steps: 0
    epochs: 10
    gradient_accumulation_steps: 1
    dataset_name: 'PDS2'
environment:
    image: "hugcyrill/workshops:chat_0.1"
records_per_epoch: 147 # 4696 examples, 32 per batch: 4696/21 is 147 records for an epoch
resources:
    slots_per_trial: 1
min_validation_period:
  batches: 4
min_checkpoint_period:
  batches: 147
searcher:
    name: adaptive_asha
    metric: eval_loss
    max_length:
        epochs: 30
    smaller_is_better: true
    max_trials: 4
max_restarts: 0
entrypoint: model_def:GPT2Finetune

```

# User Task: Update `const_ds_chatbot.yaml` and `const_eng_to_latex.yaml` configs
Then, please  the `const_ds_chatbot.yaml` file in the `determined_files/` folder and look for the workspace and project fields. Please replace the placeholders with your workspace name and project name, which you created during the preparation session:

```yaml
workspace: <your_workspace>
project: <your_project>
```

Update the `const_eng_to_latex.yaml` file in the `determined_files/`in the same manner.  replace the placeholders with your workspace name and project name,

# Finetuning Chatbot Part 1: Data Science Chatbot

In [8]:
!det experiment create \
    determined_files/const_ds_chatbot.yaml \
    determined_files/ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Preparing files to send to master... 10.5KB and 8 files
Created experiment 59


### See Result

In [7]:
experiment_id = 66
MODEL_NAME = "gpt2"
checkpoint = client.get_experiment(experiment_id).top_checkpoint(sort_by="eval_loss", smaller_is_better=True)
print(checkpoint.uuid)
loaded_model = load_model_from_checkpoint(checkpoint)

6a01f70b-55c3-4e10-874c-6684d8425b20


In [8]:

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200,  'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

Here is how the chatbot responds to same prompt after finetuning

In [9]:
PROMPT='A test statistic is a value'

In [10]:
print('----------')
for generated_sequence in finetuned_generator(PROMPT, num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


----------
A test statistic is a value that represents (what) the test-takers used to be. Let's get an idea for the types of data
data.

We should be aware that the best way to understand the data that the test
----------
A test statistic is a value at the 0% level and refers to an estimate made that is 1/10th or 0.01%
This is the
average level difference between the two levels within the same sample
The level of significance

----------
A test statistic is a value that can be seen when plotting the two data points with each other. If we want to work through the differences between our tests, we can
scatter through data,

and compare them.

[ 214
----------


# Finetuning Experiment number 2: # English 2 Latex

## Baseline english to latex chatbot
Here we are downloading pretrained weights and seeing how GPT does on a question to convert an english description into latex

In [21]:
# Sanity check that a non-finetuned model could not have done this
MODEL='gpt2'
non_finetuned_latex_generator = pipeline(
    'text-generation', 
    model=GPT2LMHeadModel.from_pretrained(MODEL),  # not fine-tuned!
    tokenizer=tokenizer
)

In [26]:
# Add our singular prompt
CONVERSION_PROMPT = 'LCT\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'
# text_sample = 'f of x is sum from 0 to x of x squared'
text_sample='x to the fourth power'
# text_sample = 'f of x equals x squared'
conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'

print(conversion_text_sample)
print(non_finetuned_latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: x to the fourth power
LaTeX:
LCT
English: x to the fourth power
LaTeX: x to the fifth power

LaTeX: x to the sixth power


## Dataset to train
To finetune the model on this niche task. We need a dataset, here we will see the dataset and how we are preprocessing it for finetuning.

In [15]:
data = pd.read_csv('./data/english_to_latex.csv')

print(data.shape)

data.head(2)

(50, 2)


,English,LaTeX
0,integral from a to b of x squared,"\int_{a}^{b} x^2 \,dx"
1,integral from negative 1 to 1 of x squared,"\int_{-1}^{1} x^2 \,dx"


This is how the dataset is preprocessed for GPT to learn via prompting

In [29]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

# Add our singular prompt
CONVERSION_PROMPT = 'LCT\n'  # LaTeX conversion task

CONVERSION_TOKEN = 'LaTeX:'
# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}English: ' + data['English'] + '\n' + CONVERSION_TOKEN + ' ' + data['LaTeX'].astype(str)

print(training_examples[0])

## Finetune
Now we have the dataset preprocessed, we will now finetune the model.

In [17]:
!det experiment create \
    determined_files/const_eng_to_latex.yaml \
    determined_files/ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Preparing files to send to master... 10.5KB and 8 files
Created experiment 60


from determined.experimental import client
# Lets see how trained checkpoint performs

In [15]:
# Get the best checkpoint from the training
experiment_id = 71
MODEL_NAME = "gpt2"
checkpoint = client.get_experiment(experiment_id).top_checkpoint(sort_by="eval_loss", smaller_is_better=True)
print(checkpoint.uuid)
loaded_model = load_model_from_checkpoint(checkpoint)


30eab3c5-4ea7-48a5-9873-e09a2c785336


In [16]:

latex_generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

# Extra: few shot prompting

In [27]:
# text_sample = 'f of x equals integral from 0 to pi of x to the fourth power'
text_sample = 'x to the fourth power'

conversion_text_sample = f'{CONVERSION_PROMPT}English: {text_sample}\n{CONVERSION_TOKEN}'
print(latex_generator(
    conversion_text_sample, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: x to the fourth power
LaTeX: x^4 * \pi^4 \,dx
LaTeX: x^4 * \


# Lets see hyperparameter search at scale

In [ ]:
!det experiment create determined_files/adaptive_ds_chatbot.yaml determined_files/

# Extra knowledge to improve Generative Modeling at Inference time: Few Shot prompting
Here we include examples of correct conversions for GPT for additional context

In [28]:
few_shot_prompt = """LCT
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx \
###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx \
###
LCT
English: f of x is x squared
LaTeX:"""

In [22]:
print(latex_generator(
    few_shot_prompt, num_beams=5, early_stopping=True, temperature=0.7,
    max_length=len(tokenizer.encode(few_shot_prompt)) + 20
)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LCT
English: f of x is sum from 0 to x of x squared
LaTeX: f(x) = \sum_{0}^{x} x^2 \,dx ###
LCT
English: f of x equals integral from 0 to pi of x to the fourth power
LaTeX: f(x) = \int_{0}^{\pi} x^4 \,dx ###
LCT
English: f of x is x squared
LaTeX: f(x) = \int_{0}^{\pi} x^2 \,dx


# User Exercise 1: Finetune book to author of choice
Lets form into groups, download text from online, and train our own chatbot!

Steps to integrate custom dataset:
* Go to project gutenburg and pick a book: (i.e. https://www.gutenberg.org/ebooks/1787 )
* copy URL Plain Text UTF-8 .txt file and download using command: 
    - Example: `wget -O hamlet.txt https://www.gutenberg.org/cache/epub/1787/pg1787.txt`
* move to shared directory: `cp hamlet.txt /run/determined/workdir/shared_fs/exercise/ -v`
* Copy `run_det_ds_chatbot.sh` and rename: 
    - i.e. `cp determined_files/const_ds_chatbot.yaml determined_files/const_hamlet_chatbot.yaml `
* NOTE: MAKE SURE THAT the file is a text file, and that there are no spaces in the name of the text file
* Finally, change `name` field that describes the name of the experiment
    - example -> `name: gpt2_finetune_hamlet_chatbot`
* Change the `dataset_name` to name of text file: (i.e. hamlet)
    - Do not include `.txt` in dataset name

In [50]:
!wget -O <CHANGE_NAME>.txt <URL_TO_TXT_FILE>

In [48]:
!cp <CHANGE_NAME>.txt /run/determined/workdir/shared_fs/workshop_data/ -v

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
'hamlet.txt' -> '/run/determined/workdir/shared_fs/workshop_data/hamlet.txt'


In [ ]:
!cp determined_files/const_ds_chatbot.yaml determined_files/const_<CHANGE_NAME>_chatbot.yaml

Edit the following fields in `const_<CHANGE_NAME>_chatbot.yaml`
```yaml
name: <CHANGE_EXPERIMENT_NAME>
workspace: <your_workspace>
project: <your_project>
description: "Chicago DS Workshop"
hyperparameters:
    global_batch_size: 32
    weight_decay: 0.0
    learning_rate: 5e-5
    adam_epsilon: 1e-8
    warmup_steps: 0
    epochs: 10
    device: 'cuda'
    gradient_accumulation_steps: 1
    dataset_name: <DATASET_NAME_TO_NEW_NAME>
    train_batch_size: 32
    eval_batch_size: 32
environment:
    image: "hugcyrill/workshops:chat_0.1"
records_per_epoch: 147 # 4696 examples, 128 per batch: 601088/21 is 147 records for an epoch
resources:
    slots_per_trial: 1
min_validation_period:
  batches: 4
min_checkpoint_period:
  batches: 147
searcher:
    name: single
    metric: eval_loss
    max_length:
        epochs: 30
    smaller_is_better: true
max_restarts: 0
entrypoint: model_def:GPT2Finetune
```

In [49]:
# Run finetuning job 
!det experiment create determined_files/const_<CHANGE_ME>_chatbot.yaml determined_files/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Preparing files to send to master... 19.9KB and 16 files
Created experiment 74


In [40]:
# Run inference
experiment_id = 74
MODEL_NAME = "gpt2"
checkpoint = client.get_experiment(experiment_id).top_checkpoint(sort_by="eval_loss", smaller_is_better=True)
print(checkpoint.uuid)
loaded_model = load_model_from_checkpoint(checkpoint)

finetuned_generator = pipeline(
    'text-generation', model=loaded_model, tokenizer=tokenizer,
    config={'max_length': 200,  'do_sample': True, 'top_p': 0.9, 'temperature': 0.7, 'top_k': 10}
)

ce16c591-252a-4d22-8fb6-9258e8c01542


In [45]:
PROMPT='My name' # Example: PROMPT='Hamlet:'

In [51]:
print('----------')
for generated_sequence in finetuned_generator(PROMPT, num_return_sequences=3):
    print(generated_sequence['generated_text'])
    print('----------')